In [66]:
import pandas as pd
from pandas.plotting import autocorrelation_plot
from sktime.forecasting.all import temporal_train_test_split
from sktime.forecasting.all import ExponentialSmoothing
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError
from sktime.forecasting.all import ForecastingHorizon
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from copy import deepcopy
import matplotlib.pyplot as plt
import numpy as np
import random

import warnings
warnings.filterwarnings('ignore')

In [38]:
train = pd.read_csv('train_msft.csv',index_col = 0)
train.index = pd.to_datetime(train.index)
train.index = pd.core.indexes.period.PeriodIndex(train.index.tolist(), freq='b')

val = pd.read_csv('val_msft.csv',index_col = 0)
val.index = pd.to_datetime(val.index)

test = pd.read_csv('test_msft.csv',index_col = 0)
test.index = pd.to_datetime(test.index)

In [39]:
train.isna().sum()
val.isna().sum()
test.isna().sum()

adj_close    0
dtype: int64

## fit model

In [40]:
def fit_smoothing(df,method='mul',period=12):
    
    fit = ExponentialSmoothing(df,
                            trend=method,
                            seasonal_periods=period, 
                            seasonal=method
                           ).fit()
    return fit

In [51]:
model = fit_smoothing(train)
y_test = val
y_pred = model.forecast(len(y_test))
y_pred.index = y_test.index

## tune parameter

In [62]:
def evaluation(method_fit,y_test):
    # Fit the model, with selected alpha value
    fit = method_fit
    # Forecast over testing period
    y_pred = fit.forecast(len(y_test))
    y_pred.index = y_test.index
    
    # Calculate smape of test set performance
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test))
    smape = np.mean(2*np.abs(y_pred - y_test) / ((np.abs(y_test) + np.abs(y_pred))))
    return [mae,mse,mape,smape]

In [81]:
def tune_param(train,val,methods=['add','mul'],periods=[4,12,24,36]):
    result = []
    
    for method in methods:
        for period in periods:
            model = fit_smoothing(train,method,period)
            evaluation_i = evaluation(model,val)

            result_i = pd.Series([method,period]+evaluation_i)
            result.append(result_i)

    result = pd.concat(result,axis=1).T
    result.columns = ['methods','periods','mae','mse','mape','smape']

    return result

In [82]:
result = tune_param(train,val['adj_close'],methods=['add','mul'],periods=[4,12,24,36])
result

,methods,periods,mae,mse,mape,smape
0,add,4,27.890538,1307.762245,0.378223,0.507648
1,add,12,27.835394,1304.497268,0.37724,0.50618
2,add,24,27.805623,1302.61346,0.376724,0.505395
3,add,36,27.894793,1308.067824,0.378293,0.50776
4,mul,4,8.647014,168.240392,0.11334,0.121621
5,mul,12,8.410525,157.730839,0.111406,0.118721
6,mul,24,8.349203,154.94908,0.110979,0.118021
7,mul,36,8.568744,165.238878,0.112394,0.120479


In [84]:
best_param = result.loc[result['smape'].idxmin(), ['methods', 'periods']].tolist()
best_param 

['mul', 24]